In [1]:
import pandas as pd
import numpy as np
from sko.GA import GA_TSP
import matplotlib.pyplot as plt
from sko.operators import ranking, selection, crossover, mutation
from itertools import product
import time
import multiprocessing as mp
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/distanceslonglat.csv')
df.head()

,Unnamed: 0,Start,Target,StartLatitude,TargetLatitude,StartLongitude,TargetLongitude,Distance
0,0,"Aberdeen, Scotland","Aberdeen, Scotland",57.15,57.150000,-2.15,-2.150000,0.000000
1,1,"Aberdeen, Scotland","Adelaide, Australia",57.15,-34.916667,-2.15,138.600000,16183.676404
2,2,"Aberdeen, Scotland","Algiers, Algeria",57.15,36.833333,-2.15,3.000000,2290.736724
3,3,"Aberdeen, Scotland","Amsterdam, Netherlands",57.15,52.366667,-2.15,4.883333,698.243287
4,4,"Aberdeen, Scotland","Ankara, Turkey",57.15,39.916667,-2.15,32.916667,3164.736857


In [3]:
cities = df['Start'].unique()
cities_index = dict(zip(cities, range(len(cities))))
df['Start'] = [cities_index[city] for city in df['Start']]
df['Target'] = [cities_index[city] for city in df['Target']]
df.head()

,Unnamed: 0,Start,Target,StartLatitude,TargetLatitude,StartLongitude,TargetLongitude,Distance
0,0,0,0,57.15,57.150000,-2.15,-2.150000,0.000000
1,1,0,1,57.15,-34.916667,-2.15,138.600000,16183.676404
2,2,0,2,57.15,36.833333,-2.15,3.000000,2290.736724
3,3,0,3,57.15,52.366667,-2.15,4.883333,698.243287
4,4,0,4,57.15,39.916667,-2.15,32.916667,3164.736857


In [4]:
start = df['Start'].to_list()
new_df = pd.DataFrame(columns=set(start))

for i in start:
    rows = df.loc[df['Start'] == i]
    distances = rows['Distance'].to_list()
    new_df.loc[i] = distances
     
distance_matrix = new_df.to_numpy()

In [5]:
def cal_total_distance(routine):
    '''The objective function. input routine, return total distance.
    cal_total_distance(np.arange(num_points))
    '''
    num_points, = routine.shape
    return sum([distance_matrix[routine[i % num_points], routine[(i + 1) % num_points]] for i in range(num_points)])

In [6]:
def genetic_tsp_variant(args, queue=None):
    num_points, max_iter, prob_mut = args
    ga_tsp = GA_TSP(func=cal_total_distance, n_dim=num_points, size_pop=50, max_iter=max_iter, prob_mut=prob_mut)
    ga_tsp.run()

    result = (ga_tsp.best_x, ga_tsp.best_y, args)

    if queue:
        queue.put(result)
    
    return result

In [7]:
num_points = [len(df['Start'].unique())]
max_iterations = [100, 500, 1000]
prob_mutations = [0.001, 0.01, 0.05]
variations = list(product(num_points, max_iterations, prob_mutations))

In [8]:
results = []
start = time.time()
for variation in tqdm(variations):
    result = genetic_tsp_variant(variation)
    results.append(result)


total = time.time() - start
results = sorted(results, key=lambda x: x[1]) 
print(f'Took total time of {total} seconds.')
print(f'Best Result was: {results[0][1]} (num_points: {results[0][2][0]} max_iter: {results[0][2][1]} prob_mut:{results[0][2][2]})')
print(f'Worst Result was: {results[-1][1]} (num_points: {results[-1][2][0]} max_iter: {results[-1][2][1]} prob_mut:{results[-1][2][2]})')

100%|██████████| 9/9 [00:29<00:00,  3.33s/it]

Took total time of 30.000304222106934 seconds.
Best Result was: [242345.39948411] (num_points: 120 max_iter: 1000 prob_mut:0.05)
Worst Result was: [735499.64360487] (num_points: 120 max_iter: 100 prob_mut:0.001)


In [9]:
results = []
start = time.time()
n_cores=mp.cpu_count()

pool = mp.Pool(n_cores)
results = pool.map(genetic_tsp_variant, variations)
pool.close()

total = time.time() - start
results = sorted(results, key=lambda x: x[1]) 
print(f'Took total time of {total} seconds.')
print(f'Best Result was: {results[0][1]} (num_points: {results[0][2][0]} max_iter: {results[0][2][1]} prob_mut:{results[0][2][2]})')
print(f'Worst Result was: {results[-1][1]} (num_points: {results[-1][2][0]} max_iter: {results[-1][2][1]} prob_mut:{results[-1][2][2]})')

Took total time of 6.442743301391602 seconds.
Best Result was: [225104.33251168] (num_points: 120 max_iter: 1000 prob_mut:0.05)
Worst Result was: [679442.75619266] (num_points: 120 max_iter: 100 prob_mut:0.001)


In [10]:
presults = []
start = time.time()
processes = []
queue = mp.Queue()
for variant in variations:
    p = mp.Process(target=genetic_tsp_variant, args=(variant, queue))
    processes.append(p)
    p.start()


for process in processes:
    process.join()

presults = [queue.get() for _ in processes]

total = time.time() - start
presults = sorted(presults, key=lambda x: x[1]) 
print(f'Took total time of {total} seconds.')
print(f'Best Result was: {presults[0][1]} (num_points: {presults[0][2][0]} max_iter: {presults[0][2][1]} prob_mut:{presults[0][2][2]})')
print(f'Worst Result was: {presults[-1][1]} (num_points: {presults[-1][2][0]} max_iter: {presults[-1][2][1]} prob_mut:{presults[-1][2][2]})')

Took total time of 6.572890996932983 seconds.
Best Result was: [225104.33251168] (num_points: 120 max_iter: 1000 prob_mut:0.05)
Worst Result was: [679442.75619266] (num_points: 120 max_iter: 100 prob_mut:0.001)
